In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn import model_selection, preprocessing, metrics
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import shap
import os
print(os.listdir("../input/ashrae-energy-prediction"))
from sklearn import preprocessing
import xgboost as xgb
import gc
import datetime
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder


import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Any results you write to the current directory are saved as output.

['test.csv', 'building_metadata.csv', 'train.csv', 'weather_test.csv', 'sample_submission.csv', 'weather_train.csv']


In [6]:
train = pd.read_csv('../input/ashrae-energy-prediction/train.csv')
#test = pd.read_csv('../input/ashrae-energy-prediction/test.csv')
building_metadata = pd.read_csv('../input/ashrae-energy-prediction/building_metadata.csv')
weather_train = pd.read_csv('../input/ashrae-energy-prediction/weather_train.csv')
#weather_test = pd.read_csv('../input/ashrae-energy-prediction/weather_test.csv')
#sample_submission = pd.read_csv('../input/ashrae-energy-prediction/sample_submission.csv')

In [7]:
train.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,1,0,2016-01-01 00:00:00,0.0
2,2,0,2016-01-01 00:00:00,0.0
3,3,0,2016-01-01 00:00:00,0.0
4,4,0,2016-01-01 00:00:00,0.0


In [8]:
train.meter_reading.values.min()

0.0

In [9]:
train.meter_reading.values.max()

21904700.0

In [10]:
weather_train.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


In [11]:
print("weather_train : air_temp : nan_sum : ", np.sum(weather_train.air_temperature.values  == np.nan))
print("weather_train : cloud_coverage : nan_sum : ", np.sum(weather_train.cloud_coverage.values  == np.nan))
print("weather_train : dew_temperature : nan_sum : ", np.sum(weather_train.dew_temperature.values  == np.nan))
print("weather_train : precip_depth_1_hr : nan_sum : ", np.sum(weather_train.precip_depth_1_hr.values  == np.nan))
print("weather_train : sea_level_pressure : nan_sum : ", np.sum(weather_train.sea_level_pressure.values  == np.nan))
print("weather_train : wind_direction : nan_sum : ", np.sum(weather_train.wind_direction.values  == np.nan))
print("weather_train : wind_speed : nan_sum : ", np.sum(weather_train.wind_speed.values  == np.nan))

weather_train : air_temp : nan_sum :  0
weather_train : cloud_coverage : nan_sum :  0
weather_train : dew_temperature : nan_sum :  0
weather_train : precip_depth_1_hr : nan_sum :  0
weather_train : sea_level_pressure : nan_sum :  0
weather_train : wind_direction : nan_sum :  0
weather_train : wind_speed : nan_sum :  0


In [12]:
building_metadata.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


In [13]:
train = train.merge(building_metadata, on='building_id', how='left')

In [14]:
train = train.merge(weather_train, on=['site_id', 'timestamp'], how='left')

In [15]:
#del building_metadata, weather_train
#gc.collect()

In [16]:
train.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
2,2,0,2016-01-01 00:00:00,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
3,3,0,2016-01-01 00:00:00,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
4,4,0,2016-01-01 00:00:00,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0


In [17]:
train['square_feet'] = np.log1p(train['square_feet'].values) # square_feet 의 값을 log(1+value) 한 값으로 변경

In [18]:
#train['year'] = pd.to_datetime(train['timestamp']).dt.year

In [19]:
#train['month'] = pd.to_datetime(train['timestamp']).dt.month#
#train['hour'] = pd.to_datetime(train['timestamp']).dt.hour
#train['dayofweek'] = pd.to_datetime(train['timestamp']).dt.dayofweek
train['weekday'] = ((pd.to_datetime(train['timestamp']).dt.dayofweek) // 5 == 1).astype(float)


In [20]:
train.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,weekday
0,0,0,2016-01-01 00:00:00,0.0,0,Education,8.913685,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,0.0
1,1,0,2016-01-01 00:00:00,0.0,0,Education,7.908755,2004.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,0.0
2,2,0,2016-01-01 00:00:00,0.0,0,Education,8.589886,1991.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,0.0
3,3,0,2016-01-01 00:00:00,0.0,0,Education,10.072639,2002.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,0.0
4,4,0,2016-01-01 00:00:00,0.0,0,Education,11.666573,1975.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,0.0


In [21]:
#le = LabelEncoder()
#le.fit(train['primary_use'].values)
#train['primary_use'] = le.transform(train['primary_use'].values)

In [22]:
#train.head()

In [23]:
#ohe_pu = OneHotEncoder()
#primary_use = ohe_pu.fit_transform(train['primary_use'].values.reshape(-1,1))

In [24]:
#ohe_m = OneHotEncoder()
#month = ohe_m.fit_transform(train['month'].values.reshape(-1,1)).toarray()
#ohe_h = OneHotEncoder()
#hour = ohe_h.fit_transform(train['hour'].values.reshape(-1,1)).toarray()
#ohe_d = OneHotEncoder()
#dayofweek = ohe_d.fit_transform(train['dayofweek'].values.reshape(-1,1)).toarray()

In [25]:
#primary_use.shape

In [26]:
#dayofweek.shape

In [27]:
y = np.log1p(train[['meter_reading']].values.reshape(-1,1))
#np.save('y', y)
train = train[['square_feet', 'year_built', 'air_temperature', 'dew_temperature', 'sea_level_pressure', 'wind_direction', 'wind_speed', 'precip_depth_1_hr', 'floor_count', 'weekday']]
#train.to_csv('new_train.csv', index=False)

train = train.values

In [28]:
#train = np.hstack([train, dayofweek])
#del dayofweek
#gc.collect()

In [29]:
train.shape

(20216100, 10)

In [30]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(train)

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='mean', verbose=0)

In [31]:
train = imp.transform(train)

In [32]:
clf = LinearRegression()
clf.fit(train, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [33]:
del train
gc.collect()

11

In [34]:
test = pd.read_csv('../input/ashrae-energy-prediction/test.csv')
weather_test = pd.read_csv('../input/ashrae-energy-prediction/weather_test.csv')
sample_submission = pd.read_csv('../input/ashrae-energy-prediction/sample_submission.csv')

In [35]:
sample_submission.head()

,row_id,meter_reading
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [36]:
test = test.merge(building_metadata, on='building_id', how='left')
test = test.merge(weather_test, on=['site_id', 'timestamp'], how='left')
del building_metadata, weather_train
gc.collect()
test['weekday'] = ((pd.to_datetime(test['timestamp']).dt.dayofweek) // 5 == 1).astype(float)
#dayofweek = ohe_d.transform(test['dayofweek'].values.reshape(-1,1)).toarray()
#test['primary_use'] = le.transform(test['primary_use'].values)
#primary_use = ohe_pu.transform(test['primary_use'].values.reshape(-1,1)).toarray()
test = test[['square_feet', 'year_built', 'air_temperature', 'dew_temperature', 'sea_level_pressure', 'wind_direction', 'wind_speed', 'precip_depth_1_hr', 'floor_count', 'weekday']]
test['square_feet'] = np.log1p(test['square_feet'].values)

#test.to_csv('new_test.csv', index=False)
test = test.values
#test = np.hstack([test, dayofweek])
#del dayofweek
#gc.collect()
test = imp.transform(test)

In [37]:
preds = np.expm1(clf.predict(test))
preds

array([[ 11.51820013],
       [  4.49506361],
       [  8.08399398],
       ...,
       [ 17.43611612],
       [ 39.89704678],
       [100.68106151]])

In [38]:
sample_submission['meter_reading'] = np.clip(preds, 0, None)
sample_submission.head()

,row_id,meter_reading
0,0,11.518200
1,1,4.495064
2,2,8.083994
3,3,30.165027
4,4,101.274005


In [39]:
sample_submission.to_csv('simple_linear_banchmar.csv', index=False)

mark